# Instagram Crawling - 제주도 여행

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
import re
import unicodedata
from tqdm.notebook import tqdm

In [3]:
chromedriver = '/Users/정현/Desktop/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

### Instagram에 접속하여 로그인

In [4]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [5]:
with open('password.txt') as f:
    password = f.read(9)

In [6]:
email = 'zhsrcd'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [7]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [8]:
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [9]:
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 제주도 맛집 검색

In [10]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [11]:
driver.get(url)
time.sleep(6)

### 첫 번째 게시글 열기

In [12]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

### 게시글 정보 가져오기

In [13]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

### 본문내용 가져오기

In [14]:
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ''
content

'.< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰보며 먹는 해물라면은 말이 필요없음!저장해뒀다가 다음 여행시 참고하기!함께 하고픈 친구소환@..#제주맛집 #제주도맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플'

### 본문에서 해시태그(#) 가져오기

In [15]:
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#제주맛집',
 '#제주도맛집',
 '#제주시맛집',
 '#애월맛집',
 '#중문맛집',
 '#제주공항맛집',
 '#서귀포맛집',
 '#산방산맛집',
 '#신제주맛집',
 '#제주시청맛집',
 '#제주애월맛집',
 '#월정리맛집',
 '#제주여행',
 '#제주핫플']

### 작성일자 정보 가져오기

In [16]:
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-18'

### 좋아요 수 가져오기

In [17]:
try:
    like = soup.select_one('.zV_Nj').text[4:-1]
except:
    like = 0
like

'1,327'

### 위치 정보 가져오기

In [18]:
try:
    place = soup.select_one('a.O4GlU').text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [19]:
row = [content, date, like, place, tags]
row

['.< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰보며 먹는 해물라면은 말이 필요없음!저장해뒀다가 다음 여행시 참고하기!함께 하고픈 친구소환@..#제주맛집 #제주도맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플',
 '2021-08-18',
 '1,327',
 '',
 ['#제주맛집',
  '#제주도맛집',
  '#제주시맛집',
  '#애월맛집',
  '#중문맛집',
  '#제주공항맛집',
  '#서귀포맛집',
  '#산방산맛집',
  '#신제주맛집',
  '#제주시청맛집',
  '#제주애월맛집',
  '#월정리맛집',
  '#제주여행',
  '#제주핫플']]

### 다음 게시글로 이동

In [20]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [21]:
# 이 다음 과정을 하기 위해 popup 화면을 없앰(즉, X를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 여러 개의 게시글 가져오기

In [22]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ''
    
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('.zV_Nj').text[4:-1]
    except:
        like = 0

    try:
        place = soup.select_one('a.O4GlU').text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [38]:
keyword = '제주여행' # 제주도맛집, 제주맛집, 제주도관광, 제주여행
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(6)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [39]:
results = []
for _ in tqdm(range(100)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(5)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(10)

  0%|          | 0/100 [00:00<?, ?it/s]

In [40]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

,content,date,like,place,tags
0,#난산리다방 #가오픈마지막주. '이번주가 마지막 입니다.'. 제가 운영하는 곳에 구...,2021-08-19,604,난산리다방,"[#난산리다방, #가오픈마지막주., #르꼬르동블루]"
1,시댁 여자들만의 점심 회동이 있던날💚주한아빠 안테 주한이 맡겨두고 아침 일찍 달려 ...,2021-08-18,951,Jeju,"[#베케, #서귀포카페]"
2,".💙 구좌 ""제주모랑"" 💙사장님들이 손수 만든 수제소품들이 가득~좋은향이 가득한 구...",2021-08-18,"1,036",,"[#제주모랑, #제주소품샵, #세화소품샵, #제주기념품, #제주기념품샵, #구좌읍소..."
3,가로사진이 훨씬 더 어울리는 공간이라 이번 피드는 좀🤔일찍 가면 홀로 명상(Mind...,2021-08-18,"1,011",월정리해변,"[#제주신상카페, #월정리카페, #아이엠더그라스]"
4,💫.초록색 가득.카페 초록달과자점제주 제주시 애월읍 납읍로2길 35-4.#드디어나도...,2021-08-19,"1,029",초록달과자점,"[#드디어나도🙌, #이런곳에서살고싶다🏡]"


In [41]:
df.tail()

,content,date,like,place,tags
93,입에서 녹아버렷,2021-08-19,58,당당 / Dang Dang,[]
94,너무나도 뜨거웠다🔥,2021-08-18,102,김녕해수욕장,[]
95,사랑에 편법은 없다,2021-08-13,745,,[]
96,언제찍었대..,2021-08-11,471,,[]
97,평화로워보이는 제주도🐎.....#jeju #마방목지 #말 #제주도 #바다 #lfl ...,2021-08-19,13,Jeju,"[#jeju, #마방목지, #말, #제주도, #바다, #lfl, #fff, #좋아요..."


In [42]:
df.to_csv('data/제주여행.csv', index=False)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 제주도맛집 / 제주맛집 / 제주도관광 / 제주여행 파일 생성

In [ ]:
# 연속적으로 파일 생성
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(6)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(2)

    results = []
    for _ in tqdm(range(200)):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(5)

    df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
    df.to_csv(f'data/{keyword}.csv', index=False)

    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(2)

In [43]:
driver.close()